In [1]:
library(edgeR)
source("formating.R")

Loading required package: limma

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, row

In [14]:
srat <- tryCatch(
    LoadSeurat('../../..//oscb/user_storage/Benchmarks/facs-Bladder_1750955885784/QC/facs_Bladder_Seurat.h5ad'),
    error = function(e) {
        stop("The file format is not supported.")
    })

srat

X -> counts



An object of class Seurat 
23341 features across 1378 samples within 1 assay 
Active assay: RNA (23341 features, 0 variable features)
 2 layers present: counts, data

In [13]:
adata <- LoadAnndata('../../../oscb/user_storage/Benchmarks/facs-Bladder_1750955885784/QC/facs_Bladder_Seurat.h5ad')

adata

AnnData object with n_obs × n_vars = 1378 × 23341
    obs: 'orig.ident', 'n_counts', 'n_genes', 'nReads', 'plate.barcode', 'mouse.id', 'tissue', 'subtissue', 'FACS.selection', 'mouse.sex', 'percent.ercc', 'free_annotation', 'cell_ontology_class', 'percent.ribo', 'res.0.4', 'cluster.ids', 'cell_ontology_id', 'pct_counts_mt', 'pct_counts_rb', 'pct_counts_hb', 'percent.plat', 'RNA_snn_res.0.5', 'seurat_clusters', 'doublet_score', 'doublet_class', 'leiden', 'louvain'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'highly_variable'
    uns: 'leiden', 'louvain', 'neighbors', 'pca'
    obsm: 'X_pca', 'X_tsne', 'X_umap', 'X_umap_3D'
    varm: 'PCs'
    layers: 'raw_counts', 'scale.data'
    obsp: 'connectivities', 'distances'

In [16]:
expression_matrix <- srat[['RNA']]@counts

In [17]:
dge <- DGEList(expression_matrix + 1)

In [21]:
uq <- calcNormFactors(dge, method="upper quartile")
uq_assay <- CreateAssayObject(data = edgeR::cpm(uq))
srat[['UpperQuartile']] <- uq_assay
SaveSeurat(srat, '../../../oscb/user_storage/Benchmarks/facs-Bladder_1750955885784/QC/results/304904e6a9cf27ee8f1d1caa299b10a5/facs_Bladder_UpperQuartile.h5seurat')

ERROR: Error in match.arg(method): 'arg' should be one of “TMM”, “TMMwsp”, “RLE”, “upperquartile”, “none”


In [20]:
adata$layers["UpperQuartile"] <- t(as.data.frame(srat[['UpperQuartile']]@data))
write_h5ad(adata, '../../../oscb/user_storage/Benchmarks/facs-Bladder_1750955885784/QC/results/304904e6a9cf27ee8f1d1caa299b10a5/facs_Bladder_UpperQuartile.h5ad', compression = "gzip")